In [3]:
print(1+1)

2


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [5]:
PINECONE_API_KEY = "2bd5fd9f-2c56-42f9-aa1b-037960262fff"
PINECONE_API_ENV = "gcp-starter"


In [6]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf("data/")

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [10]:

#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:

embeddings = download_hugging_face_embeddings()


In [12]:

embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
#query_result

In [15]:
from pinecone import Pinecone

# Initialize the Pinecone client with your API key
pc = Pinecone(api_key='8a0652b4-3efa-46ee-b3c6-962d72446b32')

# Define your index name
index_name = "medical-chatbot"  # or your specific index name

# Check if the index exists
if index_name not in pc.list_indexes().names():
    # If the index does not exist, do something (e.g., create the index)
    print(f"Index '{index_name}' does not exist.")
else:
    print(f"Index '{index_name}' exists.")

Index 'medical-chatbot' exists.


In [17]:
%pip install langchain-pinecone
%pip install poetry

  Using cached pinecone_client-3.2.2-py3-none-any.whl.metadata (16 kB)
Using cached pinecone_client-3.2.2-py3-none-any.whl (215 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 5.0.1
    Uninstalling pinecone-client-5.0.1:
      Successfully uninstalled pinecone-client-5.0.1
Note: you may need to restart the kernel to use updated packages.
  Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached more_itertools-10.3.0-py3-none-any.whl.metadata (36 kB)
   ---------------------------------------- 0.0/249.9 kB ? eta -:--:--
   ------------------- -------------------- 122.9/249.9 kB 3.6 MB/s eta 0:00:01
   ------------------------ --------------- 153.6/249.9 kB 1.8 MB/s eta 0:00:01
   -------------------------------- ------- 204.8/249.9 kB 1.6 MB/s eta 0:00:01
   -------------------------------- ------- 204.8/249.9 kB 1.6 MB/s eta 0:00:01
   -----------

In [19]:
%pip install "pinecone-client[grpc]" --upgrade

  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.2.2
    Uninstalling pinecone-client-3.2.2:
      Successfully uninstalled pinecone-client-3.2.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.0.3 requires pinecone-client<4,>=3, but you have pinecone-client 5.0.1 which is incompatible.


In [20]:
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)
import os
os.environ['PINECONE_API_KEY'] = '8a0652b4-3efa-46ee-b3c6-962d72446b32'

index_name = "medical-chatbot"
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


vectorstore_from_texts = PineconeVectorStore.from_texts(
        text_chunks,
        index_name=index_name,
        embedding=embeddings
    )

 

AttributeError: 'Document' object has no attribute 'replace'

In [21]:
import os
import pinecone
from langchain.vectorstores import Pinecone as PineconeVectorStore

# Set the Pinecone API key
api_key = '8a0652b4-3efa-46ee-b3c6-962d72446b32'
environment = 'us-west1-gcp'  # Replace with your Pinecone environment

# Initialize Pinecone


# Define your index name
index_name = "medical-chatbot"
pc = pinecone.Pinecone(api_key=api_key, environment=environment)

# Check if the index already exists
 # replace 128 with the dimension of your embeddings

# Connect to the index
index = pc.Index(index_name)  # Using GRPCIndex for the latest client

# Example placeholder for the embedding function and text chunks




# Initialize the PineconeVectorStore with the index
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Create the vector store from text chunks
vectorstore_from_texts = PineconeVectorStore.from_texts(
    text_chunks,
    index_name=index_name,
    embedding=embeddings
)

TypeError: __init__() got an unexpected keyword argument 'index_name'

In [6]:
index = pc.Index(index_name)  
index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13744}},
 'total_vector_count': 13744}

In [24]:
vectorstore = PineconeVectorStore(  
    index, embeddings, text_chunks 
)  
query = "who was Benito Mussolini?"  
vectorstore.similarity_search(  
    query,  # our search query  
    k=3  # return 3 most relevant docs  
) 

TypeError: unhashable type: 'list'

In [26]:
query_embedding = embeddings.embed_query(query)

In [27]:
query_embedding = "who was Benito Mussolini?"  
vectorstore.similarity_search(  
    query_embedding,  # our search query  
    k=3  # return 3 most relevant docs  
)  

TypeError: unhashable type: 'list'

In [28]:
from langchain_pinecone import PineconeVectorStore



vectorstore_from_docs = PineconeVectorStore.from_documents(
        text_chunks,
        index_name=index_name,
        embedding=embeddings
    )

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [29]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [30]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [31]:
%pip install ctransformers==0.2.5

Note: you may need to restart the kernel to use updated packages.


In [32]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [33]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore_from_docs.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'vectorstore_from_docs' is not defined

In [15]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

In [37]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: module 'pinecone' has no attribute 'init'

In [36]:
import os
import pinecone
from langchain.vectorstores import Pinecone as PineconeVectorStore

# Set the Pinecone API key
api_key = '8a0652b4-3efa-46ee-b3c6-962d72446b32'
environment = 'us-west1-gcp'  # Replace with your Pinecone environment

# Initialize Pinecone
pinecone.init(api_key=api_key, environment=environment)

# Define your index name
index_name = "medical-chatbot"

# Check if the index already exists
if index_name not in pinecone.list_indexes():
    # Create the index if it doesn't exist
    pinecone.create_index(name=index_name, dimension=128)  # replace 128 with the dimension of your embeddings

# Connect to the index
index = pinecone.GRPCIndex(index_name)  # Using GRPCIndex for the latest client

# Example placeholder for the embedding function and text chunks



# Initialize the PineconeVectorStore with the index
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Create the vector store from text chunks
vectorstore_from_texts = PineconeVectorStore.from_texts(
    text_chunks,
    index_name=index_name,
    embedding=embeddings
)

AttributeError: module 'pinecone' has no attribute 'init'